<a href="https://colab.research.google.com/github/Saurabhbaghel/humanlike-ocr/blob/main/atomic_char_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Saurabhbaghel/humanlike-ocr

Cloning into 'humanlike-ocr'...
remote: Enumerating objects: 364, done.
remote: Counting objects: 100% (364/364), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 364 (delta 186), reused 364 (delta 186), pack-reused 0
Receiving objects: 100% (364/364), 413.53 KiB | 4.86 MiB/s, done.
Resolving deltas: 100% (186/186), done.


In [10]:
! python /content/humanlike-ocr/training_atomic_char.py

2023-04-09 18:38:05.218422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 18:38:06.353548: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_main
EPOCH 1
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antia